### 연습문제 복습
- ratings_train.txt 데이터 로드
- document 컬럼의 결측치 제거
- 텍스트 정규화(특수문자 제거, 2칸 이상의 공백 제거, 문자열 좌우 공백 제거)
- document 컬럼의 중복 데이터 제거
- 데이터프레임의 상위 1000개의 데이터만 사용
- texts -> document 컬럼의 values 대입
- labels -> label 컬럼의 values 대입
- 토큰화 (Komoran)
    - 품사 선택: NNP, NNG, VV, VA, MAG, XR
    - 불용어: '하다', '되다', '이다'
- 단어 사전 생성
    - 사전 이름 vocab
    - 단어들의 최소 출현 횟수 2회
    - 단어 사전 생성 시 \<PAD>, \<UNK>를 가장 앞에 지정
- 토큰화된 문서들을 단어 사전의 위치값에 맞게 인코딩
    - enc_inputs 리스트에 저장

In [1]:
import pandas as pd
import numpy as np
import re
from konlpy.tag import Komoran
from collections import Counter
import torch
from sklearn.model_selection import train_test_split

ratings_train.txt 데이터 로드

In [2]:
df = pd.read_csv("../data/ratings_train.txt", sep='\t')
df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [4]:
df.isna().sum()

id          0
document    5
label       0
dtype: int64

document 컬럼의 결측치 제거

In [5]:
df.dropna(subset='document', inplace=True)
df.isna().sum()

id          0
document    0
label       0
dtype: int64

텍스트 정규화(특수문자 제거, 2칸 이상의 공백 제거, 문자열 좌우 공백 제거)

In [6]:
def normalize(text):
    text = re.sub(r"[^가-힣0-9a-zA-Z\s\.]", " ", str(text))
    text = re.sub(r"\s+", " ", text).strip()
    return text

# normalize() 함수는 데이터에 결측치 있으면 Error 발생함

방법 1. document 컬럼의 텍스트만 정규화 $\rightarrow$ Series 데이터에서 `normalize()` 함수를 실행
</br>방법 2. 여러 컬럼의 텍스트를 정규화 $\rightarrow$ DataFrame 데이터에서 `normalize()` 함수를 실행
</br>방법 3. 전체 데이터프레임을 정규화

In [7]:
# 방법 1.
# Series 데이터에서 각각의 value들을 normalize()에 대입
df['document'] = df['document'].map(normalize)
# --> 결과를 df['document']에 대입해야 함

In [8]:
# 방법 2.
# DataFrame 데이터에서 각각의 value들을 normalize()에 대입
# 'id' 컬럼은 숫자형이므로 str로 변환
# map보다 applymap 추천

# df[ ['id', 'document'] ].astype(str).applymap(normalize)

# --> 결과를 df에 대입하면 label 컬럼 사라짐
# ---> 결과를 df[ ['id', 'document'] ]에 대입해야 함

In [9]:
# 방법 3.
# 전체 데이터프레임 정규화

# df.applymap(normalize)

document 컬럼의 중복 데이터 제거

In [10]:
df.drop_duplicates(subset='document', inplace= True)
df

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요 한국인은 거들먹거리고 필리핀 혼혈은 착하다,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


데이터프레임의 상위 1000개의 데이터만 사용

In [11]:
# df = df[:1000]
df = df.head(1000).copy()

토큰화 (Komoran)
- 품사 선택: NNP, NNG, VV, VA, MAG, XR
- 불용어: '하다', '되다', '이다'

In [12]:
komoran = Komoran()

allow_pos = ['NNP', 'NNG', 'VV', 'VA', 'MAG', 'XR']
stop_word = ['하다', '되다', '이다']

def tokenize(doc):
    tokens = []
    for word, pos in komoran.pos(doc):
        if pos in allow_pos and word not in stop_word:
            tokens.append(word)
    return tokens

texts -> document 컬럼의 values 대입
</br> labels -> label 컬럼의 values 대입

In [13]:
texts, labels = df['document'].values, df['label'].values

# texts들을 이용하여 토큰화 
tokens_list = [ tokenize(doc) for doc in texts ]
tokens_list

[['더빙', '진짜', '짜증', '나', '목소리'],
 ['포스터', '초딩', '영화', '오버', '연기', '가볍'],
 [],
 ['교도소', '이야기', '솔직히', '재미', '없', '평점', '조정'],
 ['익살', '연기', '돋보이', '영화', '스파이더맨', '늙', '보이', '하', '커스틴 던스트', '너무나'],
 ['막', '걸음마', '떼', '초등학교', '학년', '용', '영화', '별', '반개', '아깝'],
 ['원작', '긴장감', '제대로', '살리'],
 ['반개',
  '아깝',
  '욕',
  '나오',
  '이응경',
  '길용우',
  '연기',
  '생활',
  '이',
  '정말',
  '발로',
  '납치',
  '감금',
  '반복',
  '반복',
  '이',
  '드라마',
  '가족',
  '없',
  '연기',
  '못하',
  '사람',
  '모이'],
 ['액션', '없', '재미', '있', '안', '영화'],
 ['왜', '평점', '낮', '꽤', '보', '헐리우드', '화려', '너무', '길들이', '있'],
 [],
 ['볼', '때', '눈물', '나서', '죽', '향수', '자극', '허진호', '감성', '절제', '멜로', '달인'],
 ['울', '손들', '횡단보도', '건너', '때', '뛰쳐나오', '이범수', '연기', '드럽'],
 ['담백', '깔끔', '좋', '신문', '기사', '로만', '보다', '보', '자꾸', '잊어버리', '사람'],
 ['취향',
  '존중',
  '진짜',
  '극장',
  '보',
  '영화',
  '가장',
  '노',
  '재',
  '노',
  '감동',
  '스토리',
  '어거지',
  '감동',
  '어거지'],
 ['매번', '긴장'],
 ['참',
  '사람',
  '웃기',
  '바스코',
  '이기',
  '락스',
  '코',
  '까',
  '고',
  '바비',
  '이기',
  '아이

In [14]:
# 토큰화된 tokens_list 에서 단어들의 출현 횟수를 확인
freq = Counter( word for toks in tokens_list for word in toks )
freq

Counter({'영화': 364,
         '보': 262,
         '하': 131,
         '없': 113,
         '있': 86,
         '좋': 73,
         '진짜': 68,
         '정말': 67,
         '안': 66,
         '너무': 61,
         '연기': 58,
         '재밌': 57,
         '같': 56,
         '나오': 52,
         '만들': 51,
         '잘': 47,
         '최고': 47,
         '다': 46,
         '사람': 44,
         '왜': 43,
         '되': 41,
         '알': 38,
         '내용': 36,
         '때': 34,
         '감동': 34,
         '말': 34,
         '더': 33,
         '재미없': 32,
         '재미': 31,
         '아깝': 31,
         '배우': 31,
         '재미있': 30,
         '생각': 30,
         '시간': 29,
         '드라마': 28,
         '그냥': 28,
         '좀': 28,
         '평점': 27,
         '스토리': 27,
         '감독': 27,
         '나': 26,
         '이': 26,
         '쓰레기': 26,
         '작품': 25,
         '모르': 24,
         '완전': 23,
         '지루': 22,
         '들': 22,
         '느낌': 21,
         '보이': 20,
         '남': 20,
         '정도': 20,
         '다시': 19,
    

In [15]:
# 참고
# Counter 사용한 코드를 풀어서 쓰면 아래와 같다.
# tokens_list에 있는 token들을 하나의 변수에 대입
t_s = []
for toks in tokens_list:
    for word in toks:
        t_s.append(word)
Counter(t_s)

Counter({'영화': 364,
         '보': 262,
         '하': 131,
         '없': 113,
         '있': 86,
         '좋': 73,
         '진짜': 68,
         '정말': 67,
         '안': 66,
         '너무': 61,
         '연기': 58,
         '재밌': 57,
         '같': 56,
         '나오': 52,
         '만들': 51,
         '잘': 47,
         '최고': 47,
         '다': 46,
         '사람': 44,
         '왜': 43,
         '되': 41,
         '알': 38,
         '내용': 36,
         '때': 34,
         '감동': 34,
         '말': 34,
         '더': 33,
         '재미없': 32,
         '재미': 31,
         '아깝': 31,
         '배우': 31,
         '재미있': 30,
         '생각': 30,
         '시간': 29,
         '드라마': 28,
         '그냥': 28,
         '좀': 28,
         '평점': 27,
         '스토리': 27,
         '감독': 27,
         '나': 26,
         '이': 26,
         '쓰레기': 26,
         '작품': 25,
         '모르': 24,
         '완전': 23,
         '지루': 22,
         '들': 22,
         '느낌': 21,
         '보이': 20,
         '남': 20,
         '정도': 20,
         '다시': 19,
    

단어 사전 생성
- 사전 이름 vocab
- 단어들의 최소 출현 횟수 2회
- 단어 사전 생성 시 \<PAD>, \<UNK>를 가장 앞에 지정

In [16]:
# 단어 사전 생성
min_count = 2

# 특수 토큰 2개를 먼저 대입
f_vocab = ["<PAD>", "<UNK>"]
b_vocab = []

# 방법 1
for word, cnt in freq.items():
    # cnt(출현 횟수)가 min_count(최소 출현 횟수)보다 크거나 같은 경우
    if cnt >= min_count:
        b_vocab.append(word)

f_vocab + b_vocab

['<PAD>',
 '<UNK>',
 '더빙',
 '진짜',
 '짜증',
 '나',
 '목소리',
 '포스터',
 '초딩',
 '영화',
 '오버',
 '연기',
 '가볍',
 '이야기',
 '솔직히',
 '재미',
 '없',
 '평점',
 '돋보이',
 '늙',
 '보이',
 '하',
 '너무나',
 '막',
 '떼',
 '초등학교',
 '학년',
 '용',
 '별',
 '반개',
 '아깝',
 '원작',
 '긴장감',
 '제대로',
 '살리',
 '욕',
 '나오',
 '생활',
 '이',
 '정말',
 '발로',
 '반복',
 '드라마',
 '가족',
 '못하',
 '사람',
 '액션',
 '있',
 '안',
 '왜',
 '낮',
 '꽤',
 '보',
 '헐리우드',
 '화려',
 '너무',
 '볼',
 '때',
 '눈물',
 '나서',
 '죽',
 '향수',
 '자극',
 '감성',
 '절제',
 '멜로',
 '울',
 '드럽',
 '담백',
 '깔끔',
 '좋',
 '기사',
 '보다',
 '자꾸',
 '취향',
 '극장',
 '가장',
 '노',
 '재',
 '감동',
 '스토리',
 '어거지',
 '참',
 '웃기',
 '이기',
 '코',
 '까',
 '고',
 '깔',
 '그냥',
 '난',
 '이해',
 '뒤',
 '갈수록',
 '재미없',
 '이건',
 '깨알',
 '캐스팅',
 '내용',
 '구성',
 '잘',
 '러',
 '드',
 '위하',
 '착하',
 '절대',
 '뜻',
 '웃',
 '가능',
 '지루',
 '같',
 '음식',
 '만찬',
 '넘',
 '별로',
 '평범',
 '수작',
 '드리',
 '주제',
 '중반',
 '다',
 '납득',
 '그렇',
 '꼭',
 '걸',
 '끄',
 '고추',
 '발',
 '센스',
 '연출력',
 '탁월',
 '90년대',
 '포스',
 '다시',
 '깨닫',
 '남',
 '꽃',
 '완전',
 '졸',
 '쓰레기',
 '시간',
 '재밌',
 '별점',
 '이리',
 '기대',
 '

In [17]:
# 방법 2
vocab = ['<PAD>', '<UNK>'] + [ word for word, cnt in freq.items() if cnt >= min_count ]
vocab

['<PAD>',
 '<UNK>',
 '더빙',
 '진짜',
 '짜증',
 '나',
 '목소리',
 '포스터',
 '초딩',
 '영화',
 '오버',
 '연기',
 '가볍',
 '이야기',
 '솔직히',
 '재미',
 '없',
 '평점',
 '돋보이',
 '늙',
 '보이',
 '하',
 '너무나',
 '막',
 '떼',
 '초등학교',
 '학년',
 '용',
 '별',
 '반개',
 '아깝',
 '원작',
 '긴장감',
 '제대로',
 '살리',
 '욕',
 '나오',
 '생활',
 '이',
 '정말',
 '발로',
 '반복',
 '드라마',
 '가족',
 '못하',
 '사람',
 '액션',
 '있',
 '안',
 '왜',
 '낮',
 '꽤',
 '보',
 '헐리우드',
 '화려',
 '너무',
 '볼',
 '때',
 '눈물',
 '나서',
 '죽',
 '향수',
 '자극',
 '감성',
 '절제',
 '멜로',
 '울',
 '드럽',
 '담백',
 '깔끔',
 '좋',
 '기사',
 '보다',
 '자꾸',
 '취향',
 '극장',
 '가장',
 '노',
 '재',
 '감동',
 '스토리',
 '어거지',
 '참',
 '웃기',
 '이기',
 '코',
 '까',
 '고',
 '깔',
 '그냥',
 '난',
 '이해',
 '뒤',
 '갈수록',
 '재미없',
 '이건',
 '깨알',
 '캐스팅',
 '내용',
 '구성',
 '잘',
 '러',
 '드',
 '위하',
 '착하',
 '절대',
 '뜻',
 '웃',
 '가능',
 '지루',
 '같',
 '음식',
 '만찬',
 '넘',
 '별로',
 '평범',
 '수작',
 '드리',
 '주제',
 '중반',
 '다',
 '납득',
 '그렇',
 '꼭',
 '걸',
 '끄',
 '고추',
 '발',
 '센스',
 '연출력',
 '탁월',
 '90년대',
 '포스',
 '다시',
 '깨닫',
 '남',
 '꽃',
 '완전',
 '졸',
 '쓰레기',
 '시간',
 '재밌',
 '별점',
 '이리',
 '기대',
 '

토큰화된 문서들을 단어 사전의 위치값에 맞게 인코딩
- enc_inputs 리스트에 저장

In [18]:
# 토큰화된 문서들을 인코딩
# 방법 1
stoi = { word:idx for idx, word in enumerate(vocab)}
stoi

{'<PAD>': 0,
 '<UNK>': 1,
 '더빙': 2,
 '진짜': 3,
 '짜증': 4,
 '나': 5,
 '목소리': 6,
 '포스터': 7,
 '초딩': 8,
 '영화': 9,
 '오버': 10,
 '연기': 11,
 '가볍': 12,
 '이야기': 13,
 '솔직히': 14,
 '재미': 15,
 '없': 16,
 '평점': 17,
 '돋보이': 18,
 '늙': 19,
 '보이': 20,
 '하': 21,
 '너무나': 22,
 '막': 23,
 '떼': 24,
 '초등학교': 25,
 '학년': 26,
 '용': 27,
 '별': 28,
 '반개': 29,
 '아깝': 30,
 '원작': 31,
 '긴장감': 32,
 '제대로': 33,
 '살리': 34,
 '욕': 35,
 '나오': 36,
 '생활': 37,
 '이': 38,
 '정말': 39,
 '발로': 40,
 '반복': 41,
 '드라마': 42,
 '가족': 43,
 '못하': 44,
 '사람': 45,
 '액션': 46,
 '있': 47,
 '안': 48,
 '왜': 49,
 '낮': 50,
 '꽤': 51,
 '보': 52,
 '헐리우드': 53,
 '화려': 54,
 '너무': 55,
 '볼': 56,
 '때': 57,
 '눈물': 58,
 '나서': 59,
 '죽': 60,
 '향수': 61,
 '자극': 62,
 '감성': 63,
 '절제': 64,
 '멜로': 65,
 '울': 66,
 '드럽': 67,
 '담백': 68,
 '깔끔': 69,
 '좋': 70,
 '기사': 71,
 '보다': 72,
 '자꾸': 73,
 '취향': 74,
 '극장': 75,
 '가장': 76,
 '노': 77,
 '재': 78,
 '감동': 79,
 '스토리': 80,
 '어거지': 81,
 '참': 82,
 '웃기': 83,
 '이기': 84,
 '코': 85,
 '까': 86,
 '고': 87,
 '깔': 88,
 '그냥': 89,
 '난': 90,
 '이해': 91,
 '뒤': 

In [19]:
# 방법 2
stoi2 = dict()
for idx, word in enumerate(vocab):
    stoi2[word] = idx
stoi2

{'<PAD>': 0,
 '<UNK>': 1,
 '더빙': 2,
 '진짜': 3,
 '짜증': 4,
 '나': 5,
 '목소리': 6,
 '포스터': 7,
 '초딩': 8,
 '영화': 9,
 '오버': 10,
 '연기': 11,
 '가볍': 12,
 '이야기': 13,
 '솔직히': 14,
 '재미': 15,
 '없': 16,
 '평점': 17,
 '돋보이': 18,
 '늙': 19,
 '보이': 20,
 '하': 21,
 '너무나': 22,
 '막': 23,
 '떼': 24,
 '초등학교': 25,
 '학년': 26,
 '용': 27,
 '별': 28,
 '반개': 29,
 '아깝': 30,
 '원작': 31,
 '긴장감': 32,
 '제대로': 33,
 '살리': 34,
 '욕': 35,
 '나오': 36,
 '생활': 37,
 '이': 38,
 '정말': 39,
 '발로': 40,
 '반복': 41,
 '드라마': 42,
 '가족': 43,
 '못하': 44,
 '사람': 45,
 '액션': 46,
 '있': 47,
 '안': 48,
 '왜': 49,
 '낮': 50,
 '꽤': 51,
 '보': 52,
 '헐리우드': 53,
 '화려': 54,
 '너무': 55,
 '볼': 56,
 '때': 57,
 '눈물': 58,
 '나서': 59,
 '죽': 60,
 '향수': 61,
 '자극': 62,
 '감성': 63,
 '절제': 64,
 '멜로': 65,
 '울': 66,
 '드럽': 67,
 '담백': 68,
 '깔끔': 69,
 '좋': 70,
 '기사': 71,
 '보다': 72,
 '자꾸': 73,
 '취향': 74,
 '극장': 75,
 '가장': 76,
 '노': 77,
 '재': 78,
 '감동': 79,
 '스토리': 80,
 '어거지': 81,
 '참': 82,
 '웃기': 83,
 '이기': 84,
 '코': 85,
 '까': 86,
 '고': 87,
 '깔': 88,
 '그냥': 89,
 '난': 90,
 '이해': 91,
 '뒤': 

In [20]:
# 인코딩 함수 정의
def encode(toks):
    result = [ stoi.get(word, stoi['<UNK>']) for word in toks ]
    return result

enc_inputs = [ torch.tensor(encode(toks), dtype=torch.long) for toks in tokens_list]
enc_inputs

[tensor([2, 3, 4, 5, 6]),
 tensor([ 7,  8,  9, 10, 11, 12]),
 tensor([], dtype=torch.int64),
 tensor([ 1, 13, 14, 15, 16, 17,  1]),
 tensor([ 1, 11, 18,  9,  1, 19, 20, 21,  1, 22]),
 tensor([23,  1, 24, 25, 26, 27,  9, 28, 29, 30]),
 tensor([31, 32, 33, 34]),
 tensor([29, 30, 35, 36,  1,  1, 11, 37, 38, 39, 40,  1,  1, 41, 41, 38, 42, 43,
         16, 11, 44, 45,  1]),
 tensor([46, 16, 15, 47, 48,  9]),
 tensor([49, 17, 50, 51, 52, 53, 54, 55,  1, 47]),
 tensor([], dtype=torch.int64),
 tensor([56, 57, 58, 59, 60, 61, 62,  1, 63, 64, 65,  1]),
 tensor([66,  1,  1,  1, 57,  1,  1, 11, 67]),
 tensor([68, 69, 70,  1, 71,  1, 72, 52, 73,  1, 45]),
 tensor([74,  1,  3, 75, 52,  9, 76, 77, 78, 77, 79, 80, 81, 79, 81]),
 tensor([1, 1]),
 tensor([82, 45, 83,  1, 84,  1, 85, 86, 87,  1, 84,  1, 88, 89, 86,  1, 90, 20]),
 tensor([ 1,  1, 91, 49, 92, 93, 94]),
 tensor([ 95,  39,  96,  97,   1,   1,  98,  99, 100,   1, 101,   1,  96, 102]),
 tensor([  1, 103,   1, 104, 105]),
 tensor([  1, 106,  4

In [21]:
# label 데이터들도 tensor 형태로 변환
labels_t = torch.tensor(labels, dtype=torch.long)   # 0, 1로 이루어진 데이터이므로 정수 형태로 변환

In [22]:
# 학습 데이터셋, 검증 데이터셋으로 분할
X_train, X_val, Y_train, Y_val = train_test_split(
    enc_inputs, labels_t, test_size=0.2, random_state=42, stratify=labels_t
)

In [23]:
len(X_train)
# 처음 데이터가 1000개였고 계층화했으나 800개

800

---
## 파이토치를 이용해 1차원 합성곱 학습 생성

In [24]:
import torch.nn as nn
# 데이터셋, 데이터로더
from torch.utils.data import Dataset, DataLoader
# 패딩('<PAD>') 토큰 -> 토큰의 길이를 채워주기 위한 기능
from torch.nn.utils.rnn import pad_sequence
import math

In [25]:
# 딥러닝에서 사용할 데이터를 파이토치에 맞게 변경
class TextDataset(Dataset):
    def __init__(self, xs, ys):
        # 독립 변수를 객체 안에 저장
        self.xs = xs
        # 종속 변수를 객체 안에 저장
        self.ys = ys
    def __len__(self):
        return len(self.xs)
    def __getitem__(self, idx):
        return self.xs[idx], self.ys[idx]

In [32]:
# 최대 커널의 개수 -> 몇 개까지의 단어들을 묶어서 확인할 것인가? (n-gram의 수)
# 그에 맞춰 <PAD> 개수 결정
MAX_K = 5

# collate_fn -> Dataset에서 배치만큼 데이터를 가져온 뒤 데이터의 형태를 변경할 때 사용하는 함수
def collate_fn(batch):
    xs, ys = zip(*batch)
    pad_id = stoi['<PAD>']  # 패딩의 위치값 (여기선 0)
    
    # batch -> [ [tensor([]), label ], [ tensor ([]), label ]
    
    # 패딩 토큰 채우기 1차
    # MAX_K보다 xs[i] 길이가 작다면 MAX_K 길이로 패딩 토큰을 채워준다.
    fixed = []
    for x in xs:
        if len(x) < MAX_K:
            # 필요한 패딩 토큰 개수: MAX_K - len(x)
            need = MAX_K - len(x)
            # tensor 데이터에 <PAD>를 채워준다. (결합 함수 cat())
            x = torch.cat(
                [
                    x,
                    torch.full( (need, ), pad_id, dtype=torch.long)
                ]
            )
        fixed.append(x)
    # 패딩 토큰 채우기 2차
    # xs에서 가장 긴 길이에 맞춰 나머지 xs[i]도 패딩 토큰을 추가한다.
    xs_pad = pad_sequence(
        fixed, batch_first= True, padding_value= pad_id
    )
    lengths = torch.tensor([len(x) for x in fixed], dtype=torch.long)
    return xs_pad, torch.stack(ys), lengths

train_loader = DataLoader(
    TextDataset(X_train, Y_train),
    batch_size=8,
    shuffle=True,
    collate_fn=collate_fn
)

val_loader = DataLoader(
    TextDataset(X_val, Y_val),
    batch_size=8,
    shuffle=True,
    collate_fn=collate_fn
)

학습 모델 생성 (kim CNN)
- Embedding
</br>$\rightarrow$ Conv1D(k = 3, 4, 5)
</br>$\rightarrow$ max-over-time (해당 구간에서 가장 연관성이 높은 구간 선택)
</br>$\rightarrow$ concat (kernel별 높은 구간)
</br>$\rightarrow$ Dropout
</br>$\rightarrow$ Linear

In [ ]:
# 학습 모델 생성 (kim CNN)
class TextCNN(nn.Module):
    def __init__(
            self,
            vocab_size,                 # 단어 사전의 길이
            emb_dim,                    # 임베딩 벡터의 차원 수
            num_classes,                # 분류 class의 개수
            kernel_size = (3, 4, 5),    # 묶이는 단어의 개수 목록
            num_channel = 100,          # 합성곱한 결과의 차원 개수
            pad_idx = 0,                # 패딩 토큰의 위치 (인덱스)
            dropout = 0.5               # 소실되는 데이터의 비율
    ):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim, padding_idx= pad_idx)
        
        # 1차원 합성곱 모델들 --> 3개
        self.convs = nn.ModuleList(
            [
                # nn 안의 1차원 합성곱(input으로 들어가는 채널 개수, out으로 나오는 채널 개수)
                nn.Conv1d(in_channels=emb_dim, out_channels=num_channel, kernel_size= k)
                for k in kernel_size
            ]
        )
        # 100차원의 모델이 3개가 생성되고, 출력들을 단순 결합하면 300차원이 되면서 과적합 위험
        # 일부 데이터를 소실시켜 과적합 방지
        self.dropout = nn.Dropout(dropout)
        # 선형 모델에서는 분류의 형태로 0, 1의 확률이 출력됨
        self.fc = nn.Linear(num_channel * len(kernel_size), num_classes)
        # 초기화
        self.__init_weights()

    # 벡터 초기화 함수
    def __init_weights(self):
        # 자비에르 초기화
        nn.init.xavier_uniform_(self.emb.weight)
        nn.init.xavier_uniform_(self.fc.weight)
        # 합성곱 모델을 초기화
        for conv in self.convs:
            # 비선형 구조에서 사용하는 다중 퍼셉트론 Relu()를 이용하는 경우
            # 학습이 안정되도록 카이밍 초기화 사용
            nn.init.kaiming_uniform_(conv.weight, a = math.sqrt(5))

    # 순전파
    def forward(self, x):
        # x: DataLoader를 통해 들어오는 입력 데이터
        # batch가 된 데이터들을 collate_fn에 대입하여 나온 결과를 순전파에 입력
        x = self.emb(x)
        # -> 배치 크기, 시퀀스의 길이, 임베딩 벡터의 차원 수 형태의 데이터가 생김 (시퀀스 길이 = 토큰의 길이)
        # 순서 바꾸기
        x = x.transpose(1, 2)
        # -> 배치 크기, 임베딩 벡터의 차원 수, 시퀀스의 길이

        feat_maps = []
        # 다중 구조 만들기 -> conv -> Relu -> Max
        for conv in self.convs:
            h = torch.relu(conv(x))             # 사이즈의 형태가 변경됨 -> 배치의 크기, relu의 차원 수, T' = 시퀀스의 길이-커널 사이즈+1
            # h 중 T'의 최댓값
            h = torch.max( h, dim= 2 ).values   # 배치의 크기, relu의 차원 수   # values를 붙여줘야 사이즈가 변환됨. 변환되지 않으면 밑에서 Error남
            feat_maps.append(h)
        # 열을 기준으로 feat_maps 단순 결합
        z = torch.cat(feat_maps, dim=1)     # 배치의 크기, relu의 차원 수 * len(self.conv)
        # 과적합 방지를 위해 일부 데이터를 0으로 변경
        z = self.dropout(z)
        # 선형 모델에서 예측
        logits = self.fc(z)
            
        return logits

In [34]:
# 학습의 루프 생성

# 모델 생성
model = TextCNN(
    vocab_size= len(vocab),
    emb_dim= 120,           # 차원의 수 = 열의 수
    num_classes= 2,         # 분류 class 종류의 수 (0, 1) -> 2
    kernel_size= (3, 4, 5), # 묶을 단어 개수
    num_channel= 64,        # conv1d에서 output의 차원 수
    pad_idx= stoi['<PAD>'], # 패딩 토큰의 위치
    dropout= 0.5            # (과적합 방지용) 고차원 데이터에서 손실 데이터의 비율
)

In [35]:
# 옵티마이저 설정
optimizer = torch.optim.Adam(model.parameters(), lr= 3e-3)

# 손실 함수 설정 (예측값과 실젯값의 차이를 계산하는 객체)
criterion = nn.CrossEntropyLoss()

In [36]:
# 학습, 예측하는 함수 정의
def run_epoch(loader, train = True):
    # loader: model에서 사용할 데이터
    # train: 학습 모드, 평가 모드
    if train:
        model.train()
    else:
        model.eval()
    
    total_loss = 0.0
    correct = 0
    total = 0

    # loader에서 데이터 가져오기
    for x, y, lengths in loader:
        # train 매개변수에 따라 자동 미분을 활성화할 것인가?
        with torch.set_grad_enabled(train):
            logits = model(x)
            loss = criterion(logits, y)
            # 학습 모드라면 -> 옵티마이저, 백워드
            if train:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
        total_loss += float(loss.item()) * x.size(0)
        # 예측값 -> [ 확률, 확률 ] -> 높은 확률의 위치
        preds = logits.argmax(dim=1)
        correct += int( (preds == y).sum().item() )
        total = x.size(0)
    
    mean_loss = total_loss / total
    acc = correct / total
    return mean_loss, acc

In [37]:
for epoch in range(10):
    tr_loss, tr_acc = run_epoch(train_loader, train= True)
    val_loss, val_acc = run_epoch(val_loader, train= False)

    print(f"epoch({epoch}): train loss({tr_loss :.4f}), train acc({tr_acc:.4f})")
    print(f"epoch({epoch}): val loss({val_loss :.4f}), val acc({val_acc:.4f})")

epoch(0): train loss(65.3012), train acc(61.6250)
epoch(0): val loss(11.9733), val acc(18.7500)
epoch(1): train loss(36.0162), train acc(83.6250)
epoch(1): val loss(12.6987), val acc(19.2500)
epoch(2): train loss(17.8074), train acc(91.5000)
epoch(2): val loss(16.4387), val acc(19.3750)
epoch(3): train loss(11.5867), train acc(95.6250)
epoch(3): val loss(21.8193), val acc(18.3750)
epoch(4): train loss(8.8505), train acc(95.7500)
epoch(4): val loss(31.2440), val acc(18.6250)
epoch(5): train loss(8.0122), train acc(95.5000)
epoch(5): val loss(27.7343), val acc(19.1250)
epoch(6): train loss(5.9082), train acc(97.0000)
epoch(6): val loss(28.9553), val acc(19.3750)
epoch(7): train loss(4.5932), train acc(97.2500)
epoch(7): val loss(33.4508), val acc(18.8750)
epoch(8): train loss(4.0628), train acc(97.8750)
epoch(8): val loss(32.3298), val acc(19.7500)
epoch(9): train loss(4.0857), train acc(97.6250)
epoch(9): val loss(33.4917), val acc(19.6250)


In [ ]:
# test 예측 함수

@torch.no_grad()
def predict(text):
    toks = tokenize(text)
    ids = torch.tensor(encode(toks), dtype=torch.long)
    pad_id = stoi['<PAD>']

    # 모델에서 최대 커널의 크기를 확인 
    max_k = max([m.kernel_size[0] for m in model.convs])

    if len(ids) < max_k:
        need = max_k - len(ids)
        ids  = torch.cat(
            [
                ids, 
                torch.full(
                    (need, ), pad_id, dtype=torch.long
                )
            ]
        )
    x = ids.unsqueeze(0)
    logits = model(x)
    prob = torch.softmax(logits, dim=1).squeeze(0).tolist()
    pred = int(torch.argmax(logits, dim=1).item())
    return prob, pred

p, yhat = predict( "직원의 태도가 별로였고 실망했다" )
print("예측값 : ", yhat)
print("예측 확률 : ", p)